In [ ]:


# 1. INSTALACIÓN DE DEPENDENCIAS
# =================================================
# En un notebook nuevo, siempre debemos instalar las librerías necesarias.
print("--- Instalando Ultralytics (YOLOv8) ---")
!pip install ultralytics -q
print("✅ Instalación completa.")


# 2. IMPORTACIONES Y MONTAJE DE DRIVE
# =================================================
import os
import shutil
import torch
import traceback
from pathlib import Path
from ultralytics import YOLO
from google.colab import drive

print("\n--- Montando Google Drive ---")
# Montamos Drive para acceder al dataset y a los modelos guardados.
drive.mount('/content/drive', force_remount=True)
print("✅ Drive montado.")


# 3. VERIFICACIÓN DE GPU
# =================================================
if not torch.cuda.is_available():
    raise RuntimeError("❌ ¡ATENCIÓN! No se detectó una GPU. Deteniendo la ejecución.")
print("\n✅ GPU detectada. ¡Listos para el ajuste fino!")
# Mostramos la información de la GPU para confirmar.
!nvidia-smi


# 4. CONFIGURACIÓN Y EJECUCIÓN DEL FINE-TUNING (FASE 2)
# =================================================
print("\n--- [FASE 2] CONFIGURANDO FINE-TUNING A ALTA RESOLUCIÓN ---")

# --- PARÁMETROS Y RUTAS PARA FASE 2 ---
MODELS_DESTINATION_DIR = Path('/content/drive/MyDrive/YOLOv8_Trained_Models/')
DATASET_YAML_PATH = "/content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml"
PROJECT_NAME = 'Coffee_Seg_High_Performance'
OPTIMIZER = 'AdamW'

F1_MODEL_VARIANT = 'yolov8x-seg.pt'
F1_IMG_SIZE = 640
F1_BATCH_SIZE = -1
PREVIOUS_RUN_NAME = f'F1_{F1_MODEL_VARIANT.split("-")[0]}_img{F1_IMG_SIZE}_b{str(F1_BATCH_SIZE)}'
MODEL_TO_FINETUNE = MODELS_DESTINATION_DIR / f'best_{PREVIOUS_RUN_NAME}.pt'

if not MODEL_TO_FINETUNE.exists():
    raise FileNotFoundError(f"❌ No se encontró el modelo de la Fase 1 en {MODEL_TO_FINETUNE}. Revisa el nombre y la ruta.")

# Parámetros específicos para el Fine-Tuning
IMG_SIZE_FT = 1024
EPOCHS_FT = 75
PATIENCE_FT = 25
LEARNING_RATE_FT = 0.0005
RUN_NAME_FT = f'F2_finetune_img{IMG_SIZE_FT}_from_{PREVIOUS_RUN_NAME}'

print(f"\nConfiguración Fase 2: \n  - Modelo a Cargar: {MODEL_TO_FINETUNE.name}\n  - Nueva Res.: {IMG_SIZE_FT}px\n  - Run: {RUN_NAME_FT}")


# --- BUCLE DE FINE-TUNING ---
# Cargamos nuestro mejor modelo de la fase anterior
model_ft = YOLO(MODEL_TO_FINETUNE)

try:
    print(f"\n>>> Iniciando Fine-Tuning Fase 2: {RUN_NAME_FT} <<<")
    model_ft.train(
        data=DATASET_YAML_PATH,
        project=PROJECT_NAME,
        name=RUN_NAME_FT,
        epochs=EPOCHS_FT,
        patience=PATIENCE_FT,
        imgsz=IMG_SIZE_FT,
        batch=-1,
        optimizer=OPTIMIZER,
        lr0=LEARNING_RATE_FT,
        copy_paste=0.3,
        mixup=0.0,
        mosaic=0.0,
        exist_ok=True,
        seed=42
    )
    print(f"\n--- ✅ FASE 2 (FINE-TUNING) COMPLETADA NORMALMENTE ---")

except (Exception, KeyboardInterrupt) as e:
    print(f"\n--- 🛑 ENTRENAMIENTO FASE 2 INTERRUMPIDO O FALLIDO: {type(e).__name__} ---")
    if not isinstance(e, KeyboardInterrupt):
        traceback.print_exc()

finally:
    print("\n--- Guardando respaldo de Fase 2... ---")
    try:
        source_model_path = Path(f'./{PROJECT_NAME}/{RUN_NAME_FT}/weights/best.pt')
        destination_model_path = MODELS_DESTINATION_DIR / f'best_{RUN_NAME_FT}.pt'
        if source_model_path.exists():
            shutil.copy2(source_model_path, destination_model_path)
            print(f"✅ ¡Éxito! Modelo final guardado en: {destination_model_path}")
        else:
            print("🟡 AVISO: No se encontró 'best.pt' en Fase 2.")
    except Exception as final_e:
        print(f"❌ Error durante guardado de Fase 2: {final_e}")

print("\n--- FIN DEL PROCESO FASE 2 ---")

--- Instalando Ultralytics (YOLOv8) ---
✅ Instalación completa.

--- Montando Google Drive ---
Mounted at /content/drive
✅ Drive montado.

✅ GPU detectada. ¡Listos para el ajuste fino!
Sat Jun 21 14:30:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |    

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              


  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640, 640, 3, 2]              
  8                  -1  3   6969600  ultralytics.nn.modules.block.C2f             [640, 640, 3, True]           
  9                  -1  1   1025920  ultralytics.nn.modules.block.SPPF            [640, 640, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  3   7379200  ultralytics.nn.modules.block.C2f             [1280, 640, 3]                
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 5.35M/5.35M [00:00<00:00, 63.9MB/s]


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  11%|█         | 1/9 [00:00<00:01,  5.21it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.03it/s]


                   all         70       1414      0.836      0.818      0.864       0.83      0.838      0.822      0.868       0.82
              Inmaduro         60        644       0.92      0.924      0.959      0.924       0.92      0.924      0.959      0.904
                Pinton         23        214      0.879      0.913      0.966      0.936      0.879      0.913      0.966      0.931
                Maduro         26        306      0.921      0.648      0.848      0.802      0.921      0.648      0.848      0.779
           Sobremaduro         18         63      0.503      0.658       0.58      0.561      0.515      0.674      0.593      0.562
                  Seco         57        187      0.957      0.949      0.969      0.929      0.957      0.949      0.972      0.925
Speed: 1.1ms preprocess, 13.9ms inference, 0.1ms loss, 6.9ms postprocess per image
Results saved to Coffee_Seg_High_Performance/F2_finetune_img1024_from_F1_yolov8x_img640_b-1

--- ✅ FASE 2 (FINE-TUNING)